# Tree Explorations

In [18]:
# General imports
import numpy as np

# set the path
import sys, os

pathArr = os.getcwd().split("/")
scriptPath = '/'.join(map(str, pathArr[:len(pathArr)-1]))
sys.path.append(scriptPath)
IMAGE_SIZE = 96
# import my tools
from tools import save4later, submit, getdata

In [6]:
from sklearn import tree
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier 
from sklearn.ensemble import AdaBoostClassifier 

In [3]:
# load the data
_loaded = getdata.load_data(0, test=True, nonas=True)

FEATURES = _loaded['features']
print 'Number of features:', len(FEATURES)

train_data = _loaded['training']['data']
train_labels = _loaded['training']['labels']
print 'Training dataset size: ', train_data.shape

test_data = _loaded['test']['data']
print 'Test dataset size: ', test_data.shape

Number of features: 30
Training dataset size:  (2140,)
Test dataset size:  (1783,)


In [13]:
%%time
dt = DecisionTreeRegressor(criterion="mse", splitter="best", random_state=0, max_features=100)
dt.fit(train_data.tolist(), train_labels[:,0])
#print 'Accuracy (a decision tree):', dt.score(train_data, test_labels)

CPU times: user 308 ms, sys: 52.9 ms, total: 361 ms
Wall time: 486 ms


In [19]:
def get_accuracy(models_list, verbose=False, ret_acc=True):
    ''' Calculates the accuracy for a given suite of models '''
    if verbose:
        print "{:30} Accuracy".format("Model")
    
    acc_list = []
    
    for index,(feat,model) in enumerate(models_list):
        predications = model.predict(train_data.tolist())
        accuracy = np.mean(1 - abs(train_labels[:,index] - predications)/ IMAGE_SIZE)
        acc_list.append(accuracy)

        if verbose:
            print " - {f:<27} {a:.3%}".format(f=FEATURES[index],a=accuracy)
    
    if ret_acc:
        return acc_list

In [20]:
models_list = []
models_list.append([FEATURES[0],dt])
get_accuracy(models_list, verbose=True, ret_acc=False)

Model                          Accuracy
 - left_eye_center_x           100.000%


In [ ]:
%%time

rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(train_data.tolist(), train_labels)

print 'Accuracy (a random forest):', rfc.score(test_data, test_labels)

In [ ]:
%%time

abc = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1), n_estimators=100, learning_rate=0.1)

abc.fit(train_data.tolist(), train_labels)
print 'Accuracy (adaboost with decision trees):', abc.score(test_data, test_labels)